### Loading Libraries

In [367]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

### Loading Datasets

In [368]:
test = pd.read_csv("../data/test_lAUu6dG.csv")
train = pd.read_csv("../data/train_ctrUa4K.csv")

In [369]:
train.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


#### Checking Counts

Dataset contains null values. 

In [370]:
train.count()

Loan_ID              614
Gender               601
Married              611
Dependents           599
Education            614
Self_Employed        582
ApplicantIncome      614
CoapplicantIncome    614
LoanAmount           592
Loan_Amount_Term     600
Credit_History       564
Property_Area        614
Loan_Status          614
dtype: int64

In [371]:
test.count()

Loan_ID              367
Gender               356
Married              367
Dependents           357
Education            367
Self_Employed        344
ApplicantIncome      367
CoapplicantIncome    367
LoanAmount           362
Loan_Amount_Term     361
Credit_History       338
Property_Area        367
dtype: int64

#### Checking null value counts

In [372]:
train.isna().sum() ##missing fields in gender, married, self employed (var) and dependents, loanamount, loan amount term, credit history

Loan_ID               0
Gender               13
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64

In [373]:
test.isna().sum()

Loan_ID               0
Gender               11
Married               0
Dependents           10
Education             0
Self_Employed        23
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount            5
Loan_Amount_Term      6
Credit_History       29
Property_Area         0
dtype: int64

**Checking proportion of null values** 

Self Employed and Credit History have more than 5% data missing. Imputing a mean or mode value can create bias in the value. We can either create a new category of unknown or impute based on subgroup

In [374]:
(100*train.isna().sum())/train.count() ##missing fields in gender, married, self employed (var) and dependents, loanamount, loan amount term, credit history

Loan_ID              0.000000
Gender               2.163062
Married              0.490998
Dependents           2.504174
Education            0.000000
Self_Employed        5.498282
ApplicantIncome      0.000000
CoapplicantIncome    0.000000
LoanAmount           3.716216
Loan_Amount_Term     2.333333
Credit_History       8.865248
Property_Area        0.000000
Loan_Status          0.000000
dtype: float64

In [375]:
(100*test.isna().sum())/test.count() ##missing fields in gender, married, self employed (var) and dependents, loanamount, loan amount term, credit history

Loan_ID              0.000000
Gender               3.089888
Married              0.000000
Dependents           2.801120
Education            0.000000
Self_Employed        6.686047
ApplicantIncome      0.000000
CoapplicantIncome    0.000000
LoanAmount           1.381215
Loan_Amount_Term     1.662050
Credit_History       8.579882
Property_Area        0.000000
dtype: float64

In [376]:
num_cols = train.select_dtypes(include=np.number).columns
var_cols = train.select_dtypes(exclude=np.number).columns

In [377]:
print(num_cols)
print(var_cols)

Index(['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History'],
      dtype='object')
Index(['Loan_ID', 'Gender', 'Married', 'Dependents', 'Education',
       'Self_Employed', 'Property_Area', 'Loan_Status'],
      dtype='object')


In [378]:
train.columns

Index(['Loan_ID', 'Gender', 'Married', 'Dependents', 'Education',
       'Self_Employed', 'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History', 'Property_Area', 'Loan_Status'],
      dtype='object')

In [379]:
# Gender               13 ##replace with mode
# Married               3 ##replace with mode
# Dependents           15 ##replace with median
# Self_Employed        32 ##replace with mode
# LoanAmount           22 ##replace with mean
# Loan_Amount_Term     14 ##replace with mode/median
# Credit_History       50 ##repace with mode

In [380]:
train.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


##### Checking if Loan Amount is changed based on a subcateogory


In [381]:
pd.pivot_table(index= 'Loan_Status',columns='Credit_History',fill_value='LoanAmount',data = train,aggfunc= 'mean')

ApplicantIncome              CoapplicantIncome               \
Credit_History             0.0          1.0               0.0          1.0   
Loan_Status                                                                  
N                  5382.841463  5613.927835       1494.719512  1786.443299   
Y                  9153.857143  5378.436508       2098.142857  1461.994497   

                LoanAmount             Loan_Amount_Term              
Credit_History         0.0         1.0              0.0         1.0  
Loan_Status                                                          
N               141.371795  155.604396       345.000000  343.051546  
Y               206.285714  142.103542       308.571429  341.967568

In [382]:
pd.pivot_table(index= 'Loan_Status',columns='Gender',fill_value='LoanAmount',data = train,aggfunc= 'mean')

ApplicantIncome              CoapplicantIncome               \
Gender               Female         Male            Female         Male   
Loan_Status                                                               
N               4091.351351  5859.240000       1682.783784  1936.226667   
Y               4915.853333  5263.814159        824.453333  1657.403894   

            Credit_History            LoanAmount             Loan_Amount_Term  \
Gender              Female      Male      Female        Male           Female   
Loan_Status                                                                     
N                 0.555556  0.543478  128.694444  157.878571       351.000000   
Y                 0.984615  0.980831  125.712329  145.612121       352.931507   

                         
Gender             Male  
Loan_Status              
N            341.793103  
Y            338.738739

In [383]:
pd.pivot_table(index= 'Loan_Status',columns='Self_Employed',fill_value='LoanAmount',data = train,aggfunc= 'mean')

ApplicantIncome              CoapplicantIncome               \
Self_Employed              No          Yes                No          Yes   
Loan_Status                                                                 
N                 5397.592357  6081.076923       1682.445860  1809.923077   
Y                 4890.530612  7984.267857       1534.550787  1358.071429   

              Credit_History        LoanAmount             Loan_Amount_Term  \
Self_Employed             No   Yes          No         Yes               No   
Loan_Status                                                                   
N                   0.540000  0.52  149.479730  156.360000       346.728477   
Y                   0.977636  1.00  138.323353  179.240741       341.715976   

                           
Self_Employed         Yes  
Loan_Status                
N              330.000000  
Y              339.333333

In [384]:
pd.pivot_table(index= 'Loan_Status',columns='Self_Employed',fill_value='LoanAmount',data = train,aggfunc= 'median')

ApplicantIncome         CoapplicantIncome      Credit_History  \
Self_Employed              No     Yes                No  Yes             No   
Loan_Status                                                                   
N                      3597.0  5117.5            1255.0  0.0            1.0   
Y                      3727.0  6034.0            1387.0  0.0            1.0   

                   LoanAmount        Loan_Amount_Term         
Self_Employed  Yes         No    Yes               No    Yes  
Loan_Status                                                   
N              1.0      127.5  155.0            360.0  360.0  
Y              1.0      124.0  147.0            360.0  360.0

In [385]:
pd.pivot_table(index= 'Loan_Status',columns='Education',fill_value='LoanAmount',data = train,aggfunc= 'mean')

ApplicantIncome              CoapplicantIncome               \
Education          Graduate Not Graduate          Graduate Not Graduate   
Loan_Status                                                               
N               6114.514286  3646.442308       2111.028571  1249.903846   
Y               5751.576471  3860.256098       1555.423294  1293.439024   

            Credit_History               LoanAmount               \
Education         Graduate Not Graduate    Graduate Not Graduate   
Loan_Status                                                        
N                 0.561538     0.489796  161.388060   122.234043   
Y                 0.980831     0.986111  151.093656   116.162500   

            Loan_Amount_Term               
Education           Graduate Not Graduate  
Loan_Status                                
N                 350.364964   326.448980  
Y                 342.340299   335.696203

In [386]:
pd.pivot_table(index= 'Loan_Status',columns='Dependents',fill_value='LoanAmount',data = train,aggfunc= 'mean')

ApplicantIncome                                         \
Dependents                0            1            2           3+   
Loan_Status                                                          
N               5071.112150  6211.305556  4073.360000  8485.833333   
Y               4848.327731  5826.439394  5207.513158  8633.242424   

            CoapplicantIncome                                         \
Dependents                  0            1            2           3+   
Loan_Status                                                            
N                 1596.943925  1832.972222  1599.040000  4290.111111   
Y                 1635.723193  1204.378788  1716.260526   788.424242   

            Credit_History                              LoanAmount  \
Dependents               0         1         2      3+           0   
Loan_Status                                                          
N                 0.588235  0.566667  0.480000  0.4375  141.831683   
Y                 0.985981  0.983333  0.985714  0.9375  133.309013   

                                                Loan_Amount_Term              \
Dependents            1           2          3+                0           1   
Loan_Status                                                                    
N            163.571429  144.217391  199.687500       353.359223  318.857143   
Y            155.873016  152.066667  186.636364       345.766234  334.909091   

                                     
Dependents            2          3+  
Loan_Status                          
N            352.800000  314.117647  
Y            336.947368  330.909091

In [387]:
pd.pivot_table(index= 'Loan_Status',columns='Property_Area',fill_value='LoanAmount',data = train,aggfunc= 'mean')

ApplicantIncome                           CoapplicantIncome  \
Property_Area           Rural    Semiurban        Urban             Rural   
Loan_Status                                                                 
N                 6497.521739  5296.851852  4511.420290       1436.507246   
Y                 4962.290909  5290.877095  5858.330827       1776.655636   

                                        Credit_History                      \
Property_Area    Semiurban        Urban          Rural Semiurban     Urban   
Loan_Status                                                                  
N              1563.203704  2565.318841           0.60  0.469388  0.538462   
Y              1507.139665  1275.908271           0.98  0.976190  0.991453   

               LoanAmount                         Loan_Amount_Term  \
Property_Area       Rural   Semiurban       Urban            Rural   
Loan_Status                                                          
N              158.742424  154.566038  140.354839       345.134328   
Y              148.261682  142.760000  143.085271       345.777778   

                                       
Property_Area   Semiurban       Urban  
Loan_Status                            
N              345.509434  341.818182  
Y              347.593220  328.186047

#### Observation

##### Impute based on subcategory for following
- Credit History is affected by Loan Status
- LOanAMount is higher for males and graduates - we can impute the mean/median of the subgroup
- loan amount term lower for self employed, not graduate and males

##### Impute based on mode and unknown for following

In [389]:
col_impute_mode = ['Gender','Married','Dependents']
col_impute_unknown = ['Self_Employed']

In [390]:
for col in col_impute_mode:
    train[col]=train[col].fillna(train[col].mode()[0])
    test[col]=test[col].fillna(test[col].mode()[0])

In [391]:
for col in col_impute_unknown:
    train[col]=train[col].fillna('Unknown')
    test[col]=test[col].fillna('Unknown')

In [392]:
impute_train_loanamt = train.groupby(['Gender','Education','Self_Employed'])['LoanAmount'].median()
impute_test_loanamt = test.groupby(['Gender','Education','Self_Employed'])['LoanAmount'].median()

impute_train_loanamt_trm = train.groupby(['Gender','Education','Self_Employed'])['Loan_Amount_Term'].median()
impute_test_loanamt_trm = test.groupby(['Gender','Education','Self_Employed'])['Loan_Amount_Term'].median()

In [393]:
impute_train_loanamt

Gender  Education     Self_Employed
Female  Graduate      No               112.5
                      Unknown          127.5
                      Yes              127.5
        Not Graduate  No               100.0
                      Unknown          115.0
                      Yes              131.5
Male    Graduate      No               134.5
                      Unknown          125.0
                      Yes              160.0
        Not Graduate  No               113.0
                      Unknown          123.0
                      Yes              130.0
Name: LoanAmount, dtype: float64

In [394]:
for i,row in train.loc[train['LoanAmount'].isnull(),:].iterrows():
    ind = tuple([row['Gender'],row['Education'],row['Self_Employed']])
    train.loc[i,'LoanAmount'] = impute_train_loanamt.loc[ind]
    
for i,row in test.loc[test['LoanAmount'].isnull(),:].iterrows():
    ind = tuple([row['Gender'],row['Education'],row['Self_Employed']])
    test.loc[i,'LoanAmount'] = impute_test_loanamt.loc[ind]

In [395]:
for i,row in train.loc[train['Loan_Amount_Term'].isnull(),:].iterrows():
    ind = tuple([row['Gender'],row['Education'],row['Self_Employed']])
    train.loc[i,'Loan_Amount_Term'] = impute_train_loanamt_trm.loc[ind]
    
for i,row in test.loc[test['Loan_Amount_Term'].isnull(),:].iterrows():
    ind = tuple([row['Gender'],row['Education'],row['Self_Employed']])
    test.loc[i,'Loan_Amount_Term'] = impute_test_loanamt_trm.loc[ind]

In [396]:
for i,row in train.loc[train['Credit_History'].isnull(),:].iterrows():
    train.loc[i,'Credit_History'] = 0 if train.loc[i,'Loan_Status'] == 'N' else 1

In [397]:
test['Credit_History']=test['Credit_History'].fillna(test['Credit_History'].mode()[0])

In [398]:
train.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,134.5,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


#### Checking if all null values are imputed

In [399]:
train.isna().sum()

Loan_ID              0
Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
Loan_Status          0
dtype: int64

In [400]:
test.isna().sum()

Loan_ID              0
Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
dtype: int64

#### Recoding values in categorical columns for encoding

In [401]:
train.Self_Employed.value_counts()

No         500
Yes         82
Unknown     32
Name: Self_Employed, dtype: int64

In [402]:
train.Self_Employed = 'self_employed_' + train.Self_Employed.astype(str)
test.Self_Employed = 'self_employed_' + test.Self_Employed.astype(str)

In [403]:
train.Credit_History.value_counts()

1.0    512
0.0    102
Name: Credit_History, dtype: int64

In [404]:
train.Credit_History = np.where(train['Credit_History'] == 1, 'Good',np.where(train['Credit_History'] == 0, 'Bad','Unknown'))
test.Credit_History = np.where(test['Credit_History'] == 1, 'Good',np.where(test['Credit_History'] == 0, 'Bad','Unknown'))

In [405]:
train.Credit_History = 'Credit_History_' + train.Credit_History.astype(str)
test.Credit_History = 'Credit_History_' + test.Credit_History.astype(str)

In [406]:
train.Credit_History.value_counts()

Credit_History_Good    512
Credit_History_Bad     102
Name: Credit_History, dtype: int64

#### Feature Engineering 

- Recoding Loan Amount term
- one hot encoding gender and married, education, self employed,  property area and loan status
- label encoding in dependents  

In [410]:
pd.pivot_table(train, index = ['Property_Area'], columns= ['Loan_Status'],aggfunc=np.sum)

ApplicantIncome         CoapplicantIncome                 \
Loan_Status                 N       Y                 N              Y   
Property_Area                                                            
Rural                  448329  545852           99119.0  195432.120001   
Semiurban              286030  947067           84413.0  269778.000000   
Urban                  311288  779158          177007.0  169695.799988   

              LoanAmount          Loan_Amount_Term           
Loan_Status            N        Y                N        Y  
Property_Area                                                
Rural            10906.0  16224.5          23844.0  38064.0  
Semiurban         8326.5  25517.5          18672.0  62244.0  
Urban             9513.5  18996.0          23640.0  43776.0

In [411]:
train.head(5)

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,self_employed_No,5849,0.0,134.5,360.0,Credit_History_Good,Urban,Y
1,LP001003,Male,Yes,1,Graduate,self_employed_No,4583,1508.0,128.0,360.0,Credit_History_Good,Rural,N
2,LP001005,Male,Yes,0,Graduate,self_employed_Yes,3000,0.0,66.0,360.0,Credit_History_Good,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,self_employed_No,2583,2358.0,120.0,360.0,Credit_History_Good,Urban,Y
4,LP001008,Male,No,0,Graduate,self_employed_No,6000,0.0,141.0,360.0,Credit_History_Good,Urban,Y


In [412]:
sorted(train.Loan_Amount_Term.unique()/12)

[1.0, 3.0, 5.0, 7.0, 10.0, 15.0, 20.0, 25.0, 30.0, 40.0]

In [413]:
sorted(train.Loan_Amount_Term.unique())

[12.0, 36.0, 60.0, 84.0, 120.0, 180.0, 240.0, 300.0, 360.0, 480.0]

In [414]:
train['Loan_Amount_Term_org'] = train.Loan_Amount_Term
test['Loan_Amount_Term_org'] = test.Loan_Amount_Term

train.Loan_Amount_Term = np.where(train['Loan_Amount_Term'] <= 60, 'Short_term_Loan',
                         np.where(train['Loan_Amount_Term'] <= 180, 'Medium_Term_Loan', 
                         np.where(train['Loan_Amount_Term'] <= 300, 'Long_Term_Loan',
                         np.where(train['Loan_Amount_Term'] <= 600, 'Very_Long_Term_Loan','Unknown'))))


test.Loan_Amount_Term =  np.where(test['Loan_Amount_Term'] <= 60, 'Short_term_Loan',
                         np.where(test['Loan_Amount_Term'] <= 180, 'Medium_Term_Loan', 
                         np.where(test['Loan_Amount_Term'] <= 300, 'Long_Term_Loan',
                         np.where(test['Loan_Amount_Term'] <= 600, 'Very_Long_Term_Loan','Unknown'))))

In [415]:
train.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,Loan_Amount_Term_org
0,LP001002,Male,No,0,Graduate,self_employed_No,5849,0.0,134.5,Very_Long_Term_Loan,Credit_History_Good,Urban,Y,360.0
1,LP001003,Male,Yes,1,Graduate,self_employed_No,4583,1508.0,128.0,Very_Long_Term_Loan,Credit_History_Good,Rural,N,360.0
2,LP001005,Male,Yes,0,Graduate,self_employed_Yes,3000,0.0,66.0,Very_Long_Term_Loan,Credit_History_Good,Urban,Y,360.0
3,LP001006,Male,Yes,0,Not Graduate,self_employed_No,2583,2358.0,120.0,Very_Long_Term_Loan,Credit_History_Good,Urban,Y,360.0
4,LP001008,Male,No,0,Graduate,self_employed_No,6000,0.0,141.0,Very_Long_Term_Loan,Credit_History_Good,Urban,Y,360.0


In [416]:
cols_to_label_enc = ['Married','Dependents']
cols_to_one_enc = ['Loan_Amount_Term','Credit_History','Property_Area','Gender','Education', 'Self_Employed','Dependents']

In [417]:
from sklearn.preprocessing import LabelEncoder

lc = LabelEncoder()

In [418]:
for col in cols_to_label_enc:
    print(col)
    print(train[col].value_counts())
    print(test[col].value_counts())
    train[col] = lc.fit_transform(train[col])
    test[col] = lc.fit_transform(test[col])
    print(train[col].value_counts())
    print(test[col].value_counts())
    print('----------------')

Married
Yes    401
No     213
Name: Married, dtype: int64
Yes    233
No     134
Name: Married, dtype: int64
1    401
0    213
Name: Married, dtype: int64
1    233
0    134
Name: Married, dtype: int64
----------------
Dependents
0     360
1     102
2     101
3+     51
Name: Dependents, dtype: int64
0     210
2      59
1      58
3+     40
Name: Dependents, dtype: int64
0    360
1    102
2    101
3     51
Name: Dependents, dtype: int64
0    210
2     59
1     58
3     40
Name: Dependents, dtype: int64
----------------


In [419]:
cols_to_label_enc = ['Loan_Status']
for col in cols_to_label_enc:
    print(col)
    print(train[col].value_counts())
    train[col] = lc.fit_transform(train[col])
    print(train[col].value_counts())
    print('----------------')

Loan_Status
Y    422
N    192
Name: Loan_Status, dtype: int64
1    422
0    192
Name: Loan_Status, dtype: int64
----------------


#### Creating new features

- Total Income
- Total people in household = applicant + coapp/spouse + dependents
- Average Income per person = Total Income / Total People in household
- Loan Amoutnt - sacle back to 1000s to compare to Income variables
- Interest Rate - instead of taking a flat interest rate, higher interest rate for riskier individuals
- EMI - based on LOan Amount and Interest rate
- EMI Expense - EMI / Total Income
- Loan Expense - Loan Amoutn / Total Income
- EMI avg - EMI / Total people in hh
- Loan avg - LOan Amount / total people in hh

In [420]:
##add new feature - avg income per depndent

train['total_hh_inc'] = train['ApplicantIncome'] + train['CoapplicantIncome']
test['total_hh_inc'] = test['ApplicantIncome'] + test['CoapplicantIncome']

In [421]:
train['total_pp_hh'] = train['Dependents'] + np.where((train['CoapplicantIncome'] > 0) , 1, 0) + np.where(train['ApplicantIncome'] > 0, 1, 0)
test['total_pp_hh'] = test['Dependents'] + np.where((test['CoapplicantIncome'] > 0) , 1, 0) + np.where(test['ApplicantIncome'] > 0, 1, 0)

In [422]:
train['avg_inc_hh_d'] = train['total_hh_inc']/train['total_pp_hh']
test['avg_inc_hh_d'] = test['total_hh_inc']/test['total_pp_hh']

In [423]:
train['LoanAmount'] = train['LoanAmount']*1000
test['LoanAmount'] = test['LoanAmount']*1000

#### Defining interest rate 

Interest rates for houses are affected by these factors 
(source: https://www.bankbazaar.com/home-loan/factors-influence-home-loan-interest-rate.html)
1. loan to value - value can be assumed to be higher in urban areas
2. job profile
3. tenure - short tenure short interest rate
4. credit score - if credit history present lower rate

In [425]:
train.columns

Index(['Loan_ID', 'Gender', 'Married', 'Dependents', 'Education',
       'Self_Employed', 'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History', 'Property_Area', 'Loan_Status',
       'Loan_Amount_Term_org', 'total_hh_inc', 'total_pp_hh', 'avg_inc_hh_d'],
      dtype='object')

In [426]:
interest_tier = train.groupby(['Self_Employed','Loan_Amount_Term','Credit_History']).median()
interest_tier

Married  \
Self_Employed         Loan_Amount_Term    Credit_History                 
self_employed_No      Long_Term_Loan      Credit_History_Bad       1.0   
                                          Credit_History_Good      1.0   
                      Medium_Term_Loan    Credit_History_Bad       1.0   
                                          Credit_History_Good      1.0   
                      Short_term_Loan     Credit_History_Good      0.0   
                      Very_Long_Term_Loan Credit_History_Bad       1.0   
                                          Credit_History_Good      1.0   
self_employed_Unknown Medium_Term_Loan    Credit_History_Bad       0.0   
                                          Credit_History_Good      1.0   
                      Very_Long_Term_Loan Credit_History_Bad       1.0   
                                          Credit_History_Good      1.0   
self_employed_Yes     Long_Term_Loan      Credit_History_Bad       1.0   
                                          Credit_History_Good      1.0   
                      Medium_Term_Loan    Credit_History_Bad       1.0   
                                          Credit_History_Good      1.0   
                      Short_term_Loan     Credit_History_Good      1.0   
                      Very_Long_Term_Loan Credit_History_Bad       0.5   
                                          Credit_History_Good      1.0   

                                                               Dependents  \
Self_Employed         Loan_Amount_Term    Credit_History                    
self_employed_No      Long_Term_Loan      Credit_History_Bad          2.0   
                                          Credit_History_Good         0.0   
                      Medium_Term_Loan    Credit_History_Bad          1.0   
                                          Credit_History_Good         1.0   
                      Short_term_Loan     Credit_History_Good         0.0   
                      Very_Long_Term_Loan Credit_History_Bad          0.0   
                                          Credit_History_Good         0.0   
self_employed_Unknown Medium_Term_Loan    Credit_History_Bad          3.0   
                                          Credit_History_Good         0.0   
                      Very_Long_Term_Loan Credit_History_Bad          1.0   
                                          Credit_History_Good         0.0   
self_employed_Yes     Long_Term_Loan      Credit_History_Bad          1.0   
                                          Credit_History_Good         2.0   
                      Medium_Term_Loan    Credit_History_Bad          0.5   
                                          Credit_History_Good         0.0   
                      Short_term_Loan     Credit_History_Good         3.0   
                      Very_Long_Term_Loan Credit_History_Bad          1.0   
                                          Credit_History_Good         0.5   

                                                               ApplicantIncome  \
Self_Employed         Loan_Amount_Term    Credit_History                         
self_employed_No      Long_Term_Loan      Credit_History_Bad            5733.5   
                                          Credit_History_Good           3814.0   
                      Medium_Term_Loan    Credit_History_Bad            3004.5   
                                          Credit_History_Good           3925.5   
                      Short_term_Loan     Credit_History_Good           3419.5   
                      Very_Long_Term_Loan Credit_History_Bad            3547.0   
                                          Credit_History_Good           3749.0   
self_employed_Unknown Medium_Term_Loan    Credit_History_Bad             416.0   
                                          Credit_History_Good           3716.0   
                      Very_Long_Term_Loan Credit_History_Bad            5000.0   
                                          Credit_History_

In [427]:
##assume base interest rate as 7% (as of 2022)
## add 1% for averse situation

In [428]:
print(train.groupby(['Property_Area'])['LoanAmount'].mean())
print(train.groupby(['Property_Area'])['LoanAmount'].median())

Property_Area
Rural        151567.039106
Semiurban    145253.218884
Urban        141136.138614
Name: LoanAmount, dtype: float64
Property_Area
Rural        134500.0
Semiurban    128000.0
Urban        120000.0
Name: LoanAmount, dtype: float64


In [460]:
#tested both median and mean - mean values have higher correlation with loan status
#train_median_loan  = train.groupby(['Property_Area'])['LoanAmount'].median()
#test_median_loan  = test.groupby(['Property_Area'])['LoanAmount'].median()

In [474]:
train_median_loan  = train.groupby(['Property_Area'])['LoanAmount'].mean()
test_median_loan  = test.groupby(['Property_Area'])['LoanAmount'].mean()

In [475]:
def calculate_interest_rate(row , median_loan):
    add_rate = 0 
    if(row.Credit_History == 'Credit_History_Bad'):
        add_rate +=3
    if(row.Self_Employed != 'self_employed_Yes'):
        add_rate +=1
    if(row.Loan_Amount_Term == 'Very_Long_Term_Loan' or row.Loan_Amount_Term == 'Long_Term_Loan'):
        add_rate +=2
    if(row.Loan_Amount_Term == 'Medium_Term_Loan'):
        add_rate +=1
    if(row.LoanAmount > median_loan[row.Property_Area]):
        add_rate +=1
    #if(row.Property_Area != 'Urban'):
    #    add_rate +=1
    
    return add_rate

In [476]:
train.loc[1,:]

Loan_ID                            LP001003
Gender                                 Male
Married                                   1
Dependents                                1
Education                          Graduate
Self_Employed              self_employed_No
ApplicantIncome                        4583
CoapplicantIncome                    1508.0
LoanAmount                         128000.0
Loan_Amount_Term        Very_Long_Term_Loan
Credit_History          Credit_History_Good
Property_Area                         Rural
Loan_Status                               0
Loan_Amount_Term_org                  360.0
total_hh_inc                         6091.0
total_pp_hh                               3
avg_inc_hh_d                    2030.333333
interest_rate                           0.1
Name: 1, dtype: object

In [477]:
calculate_interest_rate(train.loc[1,:], train_median_loan)

3

In [478]:
base_rate = 7 ##assumption based on indian house loan rates
train['interest_rate'] = train.apply(lambda x : 0.01*(base_rate + calculate_interest_rate(x,train_median_loan)), axis = 1)
test['interest_rate'] = test.apply(lambda x : 0.01*(base_rate + calculate_interest_rate(x,test_median_loan)), axis = 1)

In [479]:
train.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,Loan_Amount_Term_org,total_hh_inc,total_pp_hh,avg_inc_hh_d,interest_rate
0,LP001002,Male,0,0,Graduate,self_employed_No,5849,0.0,134500.0,Very_Long_Term_Loan,Credit_History_Good,Urban,1,360.0,5849.0,1,5849.000000,0.10
1,LP001003,Male,1,1,Graduate,self_employed_No,4583,1508.0,128000.0,Very_Long_Term_Loan,Credit_History_Good,Rural,0,360.0,6091.0,3,2030.333333,0.10
2,LP001005,Male,1,0,Graduate,self_employed_Yes,3000,0.0,66000.0,Very_Long_Term_Loan,Credit_History_Good,Urban,1,360.0,3000.0,1,3000.000000,0.09
3,LP001006,Male,1,0,Not Graduate,self_employed_No,2583,2358.0,120000.0,Very_Long_Term_Loan,Credit_History_Good,Urban,1,360.0,4941.0,2,2470.500000,0.10
4,LP001008,Male,0,0,Graduate,self_employed_No,6000,0.0,141000.0,Very_Long_Term_Loan,Credit_History_Good,Urban,1,360.0,6000.0,1,6000.000000,0.10


In [482]:
train['Loan_Status'].corr(train['interest_rate'])

-0.5258450438766701

In [484]:
train['EMI']=(train.LoanAmount*train.interest_rate*((1+train.interest_rate)**train.Loan_Amount_Term_org))/(((1+train.interest_rate)**train.Loan_Amount_Term_org)-1)
test['EMI']=(test.LoanAmount*test.interest_rate*((1+test.interest_rate)**test.Loan_Amount_Term_org))/(((1+test.interest_rate)**test.Loan_Amount_Term_org)-1)

In [485]:
train['EMI_exp']=train.EMI/train.total_hh_inc
test['EMI_exp']=test.EMI/test.total_hh_inc

In [486]:
train['loan_exp']=train.LoanAmount/train.total_hh_inc
test['loan_exp']=test.LoanAmount/test.total_hh_inc

In [487]:
train['EMI_avg']=train.EMI/train.total_pp_hh
test['EMI_avg']=test.EMI/test.total_pp_hh

In [488]:
train['loan_avg']=train.LoanAmount/train.total_pp_hh
test['loan_avg']=test.LoanAmount/test.total_pp_hh

In [489]:
from sklearn.preprocessing import OneHotEncoder

#creating instance of one-hot-encoder
encoder = OneHotEncoder()

for col in cols_to_one_enc:
    encoder_train = pd.DataFrame(encoder.fit_transform(train[[col]]).toarray())
    encoder_train.columns = encoder.categories_[0]
    train = train.join(encoder_train)
    train.drop(col, axis=1, inplace=True)


In [490]:
for col in cols_to_one_enc:
    encoder_test = pd.DataFrame(encoder.fit_transform(test[[col]]).toarray())
    encoder_test.columns = encoder.categories_[0]
    test = test.join(encoder_test)
    test.drop(col, axis=1, inplace=True)

In [491]:
train.head()

,Loan_ID,Married,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Status,Loan_Amount_Term_org,total_hh_inc,total_pp_hh,avg_inc_hh_d,...,Male,Graduate,Not Graduate,self_employed_No,self_employed_Unknown,self_employed_Yes,0,1,2,3
0,LP001002,0,5849,0.0,134500.0,1,360.0,5849.0,1,5849.000000,...,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
1,LP001003,1,4583,1508.0,128000.0,0,360.0,6091.0,3,2030.333333,...,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
2,LP001005,1,3000,0.0,66000.0,1,360.0,3000.0,1,3000.000000,...,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
3,LP001006,1,2583,2358.0,120000.0,1,360.0,4941.0,2,2470.500000,...,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
4,LP001008,0,6000,0.0,141000.0,1,360.0,6000.0,1,6000.000000,...,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0


In [492]:
test.head()

,Loan_ID,Married,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term_org,total_hh_inc,total_pp_hh,avg_inc_hh_d,interest_rate,...,Male,Graduate,Not Graduate,self_employed_No,self_employed_Unknown,self_employed_Yes,0,1,2,3
0,LP001015,1,5720,0,110000.0,360.0,5720,1,5720.000000,0.10,...,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
1,LP001022,1,3076,1500,126000.0,360.0,4576,3,1525.333333,0.10,...,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
2,LP001031,1,5000,1800,208000.0,360.0,6800,4,1700.000000,0.11,...,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,LP001035,1,2340,2546,100000.0,360.0,4886,4,1221.500000,0.10,...,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,LP001051,0,3276,0,78000.0,360.0,3276,1,3276.000000,0.10,...,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0


In [493]:
train.columns

Index([              'Loan_ID',               'Married',
             'ApplicantIncome',     'CoapplicantIncome',
                  'LoanAmount',           'Loan_Status',
        'Loan_Amount_Term_org',          'total_hh_inc',
                 'total_pp_hh',          'avg_inc_hh_d',
               'interest_rate',                   'EMI',
                     'EMI_exp',              'loan_exp',
                     'EMI_avg',              'loan_avg',
              'Long_Term_Loan',      'Medium_Term_Loan',
             'Short_term_Loan',   'Very_Long_Term_Loan',
          'Credit_History_Bad',   'Credit_History_Good',
                       'Rural',             'Semiurban',
                       'Urban',                'Female',
                        'Male',              'Graduate',
                'Not Graduate',      'self_employed_No',
       'self_employed_Unknown',     'self_employed_Yes',
                             0,                       1,
                             2,

In [362]:
train.head()

,Loan_ID,Married,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Status,Loan_Amount_Term_org,total_hh_inc,total_pp_hh,avg_inc_hh_d,...,Male,Graduate,Not Graduate,self_employed_No,self_employed_Unknown,self_employed_Yes,0,1,2,3
0,LP001002,0,5849,0.0,134500.0,1,360.0,5849.0,1,5849.000000,...,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
1,LP001003,1,4583,1508.0,128000.0,0,360.0,6091.0,3,2030.333333,...,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
2,LP001005,1,3000,0.0,66000.0,1,360.0,3000.0,1,3000.000000,...,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
3,LP001006,1,2583,2358.0,120000.0,1,360.0,4941.0,2,2470.500000,...,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
4,LP001008,0,6000,0.0,141000.0,1,360.0,6000.0,1,6000.000000,...,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0


In [497]:
train['Loan_Status'].corr(train['EMI'])

-0.13066645248571968

### Saving Clean dataset for EDA

In [499]:
train.to_csv('../data/train_cleaned.csv',index = False)
test.to_csv('../data/test_cleaned.csv',index = False)